In [ ]:
import pandas as pd

from datamender import Mender

# main_df = pd.read_csv("https://media.geeksforgeeks.org/wp-content/uploads/nba.csv")
main_df = pd.read_csv('./expermental/testing_data/titanic.csv')

main_mender = Mender(main_df)

In [4]:
main_mender.sklearn_impute(['Age'], 'iterative', remark='test1', random_state=0)
main_mender.sklearn_impute(['Age'], 'iterative', remark='test2', random_state=128, keep_empty_features=True)
main_mender.sklearn_impute(['Age'], 'knn', remark='test1', n_neighbors=2)
main_mender.sklearn_impute(['Age'], 'knn', remark='test2', n_neighbors=4)



In [ ]:
# Show all the columns
main_df.head()

In [ ]:
main_mender.overview()

In [ ]:
main_mender.insights()

In [5]:
main_mender.show_incomplete(999)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age__iterative_test1,Age__iterative_test2,Age__knn_test1,Age__knn_test2
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,22.000000,22.000000,22.0,22.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,26.000000,26.000000,26.0,26.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,35.000000,35.000000,35.0,35.0
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,28.707338,29.699118,26.5,27.5
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,2.000000,2.000000,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.0500,NaN,S,25.000000,25.000000,25.0,25.0
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q,39.000000,39.000000,39.0,39.0
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,27.000000,27.000000,27.0,27.0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,26.535386,29.699118,25.5,26.5


## Imputation Plan

Sex is object, but need to be converted to number or boolean, can use one-hot
Embarked can also be converted with one-hot

Age and cabin have most of missing value, should create a new column - Age__missing to capture the missing value in age

Cabin is too complex to process, need to ignore



In [ ]:
main_mender.df.head()

In [ ]:
# main_mender.setfocus(columns=['Survived', 'Pclass', 'Name'], rows=(0, 50))

In [ ]:
# Ignore Cabin colum because it is string value without any pattern
main_mender.setignore(columns=['Cabin'])

In [ ]:
from datamender import ImputationStrategies as im_strategy

# Apply mean imputation to the age column
# But because the age column contains too much missing data, we'll capture the likelihood of missing as new column
main_mender.impute_columns(
    columns=["Age"],
    strategies=[im_strategy.mean],
    inplace= True,
    capture_likelihood = True
)


In [ ]:
# Since there are only 2 value missed in this column
# Impute with mode since the majority takes 72.28% of the value
main_mender.impute_columns(
    columns=["Embarked"],
    strategies=[im_strategy.mode],
    inplace=True
)

In [ ]:
from datamender import EncodingStrategies as en_stragety

# Sex is object, but need to be converted to number or boolean, can use one-hot
main_mender.encode_column('Sex', en_stragety.one_hot)

In [ ]:
#Embarked can also be converted with one-hot
main_mender.encode_column('Embarked', en_stragety.one_hot)

In [ ]:
import re
def amend_height(feet):
    pattern = re.compile(r"(\d+)-(\d+)")
    if pd.isna(feet):
        return None

    found = pattern.findall(feet)
    if len(found) == 0:
        return None

    feet, inch = (found[0])
    return int(feet) * 30.48 + int(inch) * 2.54

main_mender.amend_column("Height", amend_height)

In [ ]:
main_mender.graphs(kind="kde")

In [ ]:

main_mender.setfocus(columns=['Survived'], rows=(1, 50, 2))

In [ ]:
main_mender.column_categories("Position")

In [ ]:
from datamender import EncodingStrategies as en_strategy

category_order_map = {
    "PG": 1,
    "SF": 3,
    "SG": 5,
    "PF": 7,
    "C": 100,
}

main_mender.encode_column(
    column="Position",
    strategy=en_strategy.label_encoding,
    category_order_map=category_order_map
)
